In [1]:
import gc
import itertools
import matplotlib.pyplot as plt
import numpy as np
import random

import pandas as pd
import rpy2.rinterface as rinterface
import rpy2.robjects as robjects
import tqdm
import seaborn as sns
import scipy.stats as stats

from itertools import compress
from Bio import motifs
from Bio.Seq import Seq #, IUPAC
from collections import Counter
from os import listdir
from os.path import join
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from scipy.stats import ks_2samp
from statistics import mean, median

%load_ext rpy2.ipython

%matplotlib inline
pandas2ri.activate()
plt.ioff()

In [2]:
%%R
library(Biostrings)
library("BSgenome.Hsapiens.UCSC.hg19")
library(stringi)
library(rtracklayer)
library(ggplot2)
library(patchwork)
library(gplots)
library(RColorBrewer)
library(plotly)
library(reshape2)
library("wesanderson",lib.loc="/home/kribelba/R_libs/")
library(GenomicFeatures)
library("TxDb.Hsapiens.UCSC.hg19.knownGene")
library('AnnotationHub')
library(GO.db)
library(org.Hs.eg.db)
library(ChIPseeker)
library(ggpubr)

library(viridis)
library(tidyverse)
library(dplyr)
library(DESeq2)
library(Rsubread)
library("tximport")
library("readr")
library(biomaRt)
library(ggbeeswarm)
library(ggrepel)

# general functions
makeTr <- function(someColor, alpha=100) scales::alpha(someColor, alpha/100)
source("/home/kribelba/updepla_storage/users/kribelba/scripts/Enhancer_hierarchy/GM12787_ChIP_ASB/current_version/functions_Enh_hierarchy_GM12878_mechansims_analysis_2021_11_27.R")


enh_cols = wes_palette("Rushmore1")[c(1,4,3)]

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::%within%() masks IRanges::%within%()
✖ ggplot2::Position()   masks BiocGenerics::Position(), base::Position()
✖ dplyr::collapse()     masks Biostrings::collapse(), IRanges::collapse()
✖ dplyr::combine()      masks Biobase::combine(), BiocGenerics::combine()
✖ purrr::compact()      masks XVector::compact()
✖ dplyr::desc()         masks IRanges::desc()
✖ tidyr::expand()       masks S4Vectors::expand()
✖ dplyr::filter()       masks plotly::filter(), stats::filter()
✖ dplyr::first()        masks S4Vectors::first()
✖ dplyr::ident()        masks dbplyr::ident()
✖ dplyr::lag()          masks stats::lag()
✖ purrr::reduce()       masks GenomicRanges::reduce(), IRang

RInterpreterError: Failed to parse and evaluate line 'library(Biostrings)\nlibrary("BSgenome.Hsapiens.UCSC.hg19")\nlibrary(stringi)\nlibrary(rtracklayer)\nlibrary(ggplot2)\nlibrary(patchwork)\nlibrary(gplots)\nlibrary(RColorBrewer)\nlibrary(plotly)\nlibrary(reshape2)\nlibrary("wesanderson",lib.loc="/home/kribelba/R_libs/")\nlibrary(GenomicFeatures)\nlibrary("TxDb.Hsapiens.UCSC.hg19.knownGene")\nlibrary(\'AnnotationHub\')\nlibrary(GO.db)\nlibrary(org.Hs.eg.db)\nlibrary(ChIPseeker)\nlibrary(ggpubr)\n\nlibrary(viridis)\nlibrary(tidyverse)\nlibrary(dplyr)\nlibrary(DESeq2)\nlibrary(Rsubread)\nlibrary("tximport")\nlibrary("readr")\nlibrary(biomaRt)\nlibrary(ggbeeswarm)\nlibrary(ggrepel)\n\n# general functions\nmakeTr <- function(someColor, alpha=100) scales::alpha(someColor, alpha/100)\nsource("/home/kribelba/updepla_storage/users/kribelba/scripts/Enhancer_hierarchy/GM12787_ChIP_ASB/current_version/functions_Enh_hierarchy_GM12878_mechansims_analysis_2021_11_27.R")\n\n\nenh_cols = wes_palette

## Read in alignments from Nanopore seqeuncing runs
Minion runs: 

Minion 20: 
BC14 = 3.8 library 

Minion 21:
BC19 = 3.9 library
BC20 = 4.8 library

Minion 22:
BC04 = 4.9 library

Minion23:
BC06&07 = 5.1 library

P2Solo run: 
BC08 = 5.8 library cut with SalI and NotI
BC07 = 5.7 library cut with SalI and NotI

In [6]:
%%R
path = '/path_to_alignments/'
mapfiles = list.files(path)
mapfiles[grep('tsv',mapfiles)]

[1] "3p8.nomult.tsv"          "3p9.nomult.tsv"         
[3] "4p9.nomult.tsv"          "5p1_1.nomult.tsv"       
[5] "5p1_2.nomult.tsv"        "CAATmut4p8_1.nomult.tsv"
[7] "CAATmut4p8_2.nomult.tsv"


In [114]:
%%R
# example file
m571 = read.table(paste0(path,'5p7_1.nomult.tsv'),header=T,sep='\t')
m572 = read.table(paste0(path,'5p7_2.nomult.tsv'),header=T,sep='\t')
print(head(m571))
m572$ibtw_sequence = m571$TF_sequence

print(head(m572))

  Reference                            Read_name    BC_sequence
1       ref 4e5917c7-a5fb-43d5-abee-d7dac5642cc8 CGTGTGGGAATGCA
2       ref 7d223233-422e-4c11-a782-7533fbece643   CAGGGTCCTGAG
3       ref fac8854d-6278-4600-8bc8-9f2ba7b88931   --CGTGGCGTAT
4       ref 6f788805-1600-4f83-8820-aa1218af6811   GTGTAGGAGGGG
5       ref b314856e-b870-46bb-b28c-ce7d529385a9   TTGTAGCAGGTG
6       ref fd19a940-2e0a-4cbd-8484-d0d2bd9edc91   GAGTCTGAGGCG
                            TF_sequence
1  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
2  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
3 GCGCTCGGCCGGCCCGCGCCGCCCGCCTTTAAAGGGA
4  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
5  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
6  GCGCTCGGCCGGCCCGCG-CGCCCGCCTTAAAGGGA
  Reference                            Read_name    BC_sequence
1       ref 4e5917c7-a5fb-43d5-abee-d7dac5642cc8 CGTGTGGGAATGCA
2       ref 7d223233-422e-4c11-a782-7533fbece643   CAGGGTCCTGAG
3       ref fac8854d-6278-4600-8bc8-9f2ba7b88931   --CGTGGCGTAT
4       ref 6f78

### For libraries 4p8, 5p1, and 5p7 there is a promoter and an enhancer variation that were extracted separately, so we combine them into one file

In [9]:
%%R
m511 = read.table(paste0(path,'5p1_1.nomult.tsv'),header=T,sep='\t')
m512 = read.table(paste0(path,'5p1_2.nomult.tsv'),header=T,sep='\t')
print(head(m512))
print(table(m512$Reference))
print(table(m511$Read_name == m512$Read_name))
m512$ibtw_sequence = m511$TF_sequence
print(head(m512))
write.csv(m512,'/outpath/lib5p1_BC_IBTW_TF_map_2024_07_24.csv',row.names=F)


  Reference                            Read_name  BC_sequence
1       ref 9c8aec61-298d-47df-8ada-29012375d9fc AAACTTGTCCTT
2       ref d3441c67-c0a8-4b68-9a40-db70a0f88816 AGTGGCTACTGT
3       ref 81957966-7ce8-4c5f-bb27-11f69cfdd469 GCGTTGTTGGGG
4       ref 8afb8caa-7219-4356-bf87-28f2f4965e03 CTGATAAGAACA
5       ref c5f26f4b-2582-411a-85a6-fc0738b1ae62 AAATAGGTTGAG
6       ref 802393c3-1a6e-4ad5-ab21-ad3d79753e49 -GGGTTAGGTCG
             TF_sequence               ibtw_sequence
1 ACATCTAAAATCAAAATAAACA CGCCCCTGCCACCTTAAAGGGACAGCG
2 ACATCTAAAATCAAAATAAACA   CGCCGCCCGCCTTAAAGGGACAGCG
3 ACATCTAAAATCAAAATAAACA   CGCCGCCCGCCTTAAAGGGACAGCG
4                          CGCCGCCCGCCTTAAAGGGACAGCG
5 ACATCTAAAATCAAAATAAACA   CGCCGCCCGCCTTAAAGGGACAGCG
6                        CGCACG---GCCTTAAAAGGGAAAGTC
  Reference                            Read_name  BC_sequence
1       ref 9c8aec61-298d-47df-8ada-29012375d9fc AAACTTGTCCTT
2       ref d3441c67-c0a8-4b68-9a40-db70a0f88816 AGTGGCTACTGT
3       r

In [5]:
%%R
m481 = read.table(paste0(path,'4p8_1.nomult.tsv'),header=T,sep='\t')
m482 = read.table(paste0(path,'4p8_2.nomult.tsv'),header=T,sep='\t')
print(head(m482))
print(table(m482$Reference))
print(table(m481$Read_name == m482$Read_name))
m482$ibtw_sequence = m481$TF_sequence
print(head(m482))
write.csv(m482,'/outpath/lib4p8_BC_IBTW_TF_map_2024_06_05.csv',row.names=F)

m571 = read.table(paste0(path,'5p7_1.nomult.tsv'),header=T,sep='\t')
m572 = read.table(paste0(path,'5p7_2.nomult.tsv'),header=T,sep='\t')
m572$ibtw_sequence = m571$TF_sequence
write.csv(m572,'/outpath/lib5p7_BC_IBTW_TF_map_2024_06_05.csv',row.names=F)



   Reference                            Read_name    BC_sequence
1 DPEmut_REF ecb3dfc3-a3e9-44a6-a16e-ccd4dfba370a CCGATTAGAGTGTA
2 DPEmut_REF ae3df884-28c7-4bee-8426-f4717f7c88c1 GGGGAGGGGAAAAT
3 DPEmut_REF a13e179d-fef6-43d7-bd70-59b00d77113b AAGCGGAATGAAGG
4 DPEmut_REF 395d4685-1a39-4f70-b8da-bfc349f10e19 --AGTATTGAAGAG
5 DPEmut_REF 929b3c95-f24e-45a5-9d25-cbf6ff8a6471 AAAATTTTGCTTTA
6 DPEmut_REF 3d67df6a-9608-4724-82e3-532f75ddc34c -GCAGGCCATGAGG
             TF_sequence
1 ACATCTAAAATCAAAATAAACA
2                       
3 ACATCTAAAATCAAAATAAACA
4 ACATCTAAAATCAAAATAAACA
5                       
6                       

       CAATflip_ALT      CAATflip_NFKB1        CAATflip_PU1        CAATflip_REF 
               5181                6146                6404                4735 
    CGislandDEL_ALT   CGislandDEL_NFKB1     CGislandDEL_PU1     CGislandDEL_REF 
              31319               38279               41882               28922 
         DPEmut_ALT        DPEmut_NFKB1      

In [199]:
%%R
m581 = read.table(paste0(path,'5p8_1.nomult.tsv'),header=T,sep='\t')
m582 = read.table(paste0(path,'5p8_2.nomult.tsv'),header=T,sep='\t')
print(head(m581))
m582$ibtw_sequence = m581$TF_sequence

print(head(m582))
write.csv(m582,'/outpath/lib5p8_BC_IBTW_TF_map_2024_06_05.csv',row.names=F)


  Reference                            Read_name  BC_sequence
1       ref 91e3482a-c5b8-410a-9547-58b118de5e71 GCTACGCTCCTT
2       ref e8839302-c063-4545-9ef1-6a8556e1d04e GAACCGTACGGC
3       ref d6538daf-91b6-441a-99e7-43661c04fa53 GAAGATGGGCTA
4       ref 4884527c-9787-427f-9a7e-09dd47ed96fd TCCACCTCATCG
5       ref d601abea-9e73-4f8b-90d2-3f53f6d038a1 CGTGATATGGAG
6       ref b3c19ab5-3bb3-42b6-a562-533ff0e2c372 GAATGCGCGAGG
                              TF_sequence
1 GCG-TCGGCCAAGCCACCGCGCCGCCCGCCTTAAAGGGA
2    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
3    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
4    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
5    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
6    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
  Reference                            Read_name  BC_sequence
1       ref 91e3482a-c5b8-410a-9547-58b118de5e71 GCTACGCTCCTT
2       ref e8839302-c063-4545-9ef1-6a8556e1d04e GAACCGTACGGC
3       ref d6538daf-91b6-441a-99e7-43661c04fa53 GAAGATGGGCTA
4       ref 4884527c-978

## Read in others 

In [69]:
%%R
m38 =read.table(paste0(path,'3p8.nomult.tsv'),header=T,sep='\t')

m39 =read.table(paste0(path,'3p9.nomult.tsv'),header=T,sep='\t')[,c(1,2,4,3)]

m49 =read.table(paste0(path,'4p9.nomult.tsv'),header=T,sep='\t')[,c(1,2,4,3)]
colnames(m49) = colnames(m38)
colnames(m39) = colnames(m38)

print(head(m39))



  Reference                            Read_name    BC_sequence
1      NFIA 5d7cbd7d-ac17-4f27-b2fd-1d29b8604d77 --TGAGCGAGCACG
2      NFIA af238980-268e-46e1-bcf7-45e5b07c111a GAAAAAGGCGAAAA
3      NFIA a8d62edf-385b-4d9a-8686-ec2c71c558cb GTTTTTTGGCAAGA
4      NFIA 6b2319c0-9409-4590-a87f-74e209e73000 GGATATTCTGGCAA
5      NFIA 2a99fee5-8e8a-4b5e-8349-5f6c36989fa9 TGGCCGTGTGTGGG
6      NFIA 06693176-e58d-4d2d-b9b2-f566987704ae GGATGTTCTAGGGC
         TF_sequence
1   TTGGCACGGTGCCAAA
2   -CGCCTAGGTGCCAAA
3   TTGGCACGGTGCCAAA
4   TTGGCACGGTGCCAAA
5   TTGGCACGGTGCCAAA
6 ATCGCCTA-GGTGCCAAA


## filter table function
This function will remove any barcodes that are not the correct length, or TF binding site seqeunces that are too short ot be uniquely mapped and thus ambigious. For enhancer deletion separate approach is taken.

In [200]:
%%R

filter_table = function(tab,BClen){
    fnbc = !(1:length(tab$BC_sequence) %in% grep('-',tab$BC_sequence)) & nchar(tab$BC_sequence)==BClen
    ftf = nchar(tab$TF_sequence)>6 #smallest should be 7bp for enhancer deletion position
    tabk = tab[fnbc & ftf,]
    return(tabk)
}
# for 5p7 and 5p8 bc enh del is only 1 base (should be G)
filter_table2= function(tab,BClen){
    fnbc = !(1:length(tab$BC_sequence) %in% grep('-',tab$BC_sequence)) & nchar(tab$BC_sequence)==BClen
    ftf = nchar(tab$TF_sequence)>6 |  tab$TF_sequence=='G' #enh_del should be 1bp and = G on
    tabk = tab[fnbc & ftf,]
    return(tabk)
}

# prefilter the mapping tables
m38f = filter_table(m38,14)
m39f = filter_table(m39,14)
m49f = filter_table(m49,10)
m48f = filter_table(m482,14)
m51f = filter_table(m512,12)
m57f = filter_table2(m572,12)
m58f = filter_table2(m582,12)
print(head(m58f))

  Reference                            Read_name  BC_sequence
1       ref 91e3482a-c5b8-410a-9547-58b118de5e71 GCTACGCTCCTT
2       ref e8839302-c063-4545-9ef1-6a8556e1d04e GAACCGTACGGC
3       ref d6538daf-91b6-441a-99e7-43661c04fa53 GAAGATGGGCTA
4       ref 4884527c-9787-427f-9a7e-09dd47ed96fd TCCACCTCATCG
5       ref d601abea-9e73-4f8b-90d2-3f53f6d038a1 CGTGATATGGAG
6       ref b3c19ab5-3bb3-42b6-a562-533ff0e2c372 GAATGCGCGAGG
              TF_sequence                           ibtw_sequence
1 ACATCTAAAATCAGAATAGAACA GCG-TCGGCCAAGCCACCGCGCCGCCCGCCTTAAAGGGA
2  ACATCTAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
3  ACATCTAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
4  ACAT-TAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
5  ACATCTAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
6  ACATCTAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA


In [203]:
%%R
print(head(m58f[grep('enh_del',m58f$Reference),]))


        Reference                            Read_name  BC_sequence TF_sequence
241884 wt-enh_del f2bf302b-9731-404a-abc0-aa0c4004c6d1 AAAATGGCACGG           G
241885 wt-enh_del b9a52b4e-c809-485a-b28a-33d2890d3699 AGGGCATAAAAG           G
241887 wt-enh_del c84d8f98-f493-4c0b-b8bc-b19e9262f802 TGGAGGTAGTGG           G
241888 wt-enh_del 604c59d1-21c2-43df-b8ee-b4811704f475 CGAGCGGTGGGA           G
241891 wt-enh_del 21af7046-3520-45ac-8c2f-f010a01a6b59 CTGAGTTCAAGG           G
241892 wt-enh_del 80e6cab2-3976-4593-bbe4-82ac51055fa6 TAGGCGGGGGTT           G
                               ibtw_sequence
241884  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
241885  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
241887  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
241888 GCGCTCGGCCAGCACCGCGCCGCCCGCCTTAAAGGGA
241891  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
241892 GCGCTCGGCCAGGCCCGCGCCGCCCGCCTTAAAGGGA
[1] 106


## quality filters and counting associations

In [69]:
%%R
# load BS info files
# 3p8, 4p9, 3p9, 4p8
BS_info = read.csv('/path_TFBS_info_file/RMCE_26BS_lib.csv',header=T)
BS_info$BS_seq = sapply(BS_info$seq,function(x){strsplit(x,split=" ")[[1]][2]})
print(head(BS_info))

# include the 8 TFBS in lib 3.8
TF8 = c('REF','ALT','MEF2B','PU1','IRF2','NFKB1','CTCF','FOSL1')
BS8seqs = BS_info$BS_seq[match(TF8,BS_info$TF)]
print(BS8seqs)

excl = c(TF8,'CG_rd1 ','CG_rd2','AT_rd1 ','AT_rd2','RUNX1_rc') # were not added in experiment

TF13 = setdiff(BS_info$TF,excl) # TFs in library 3.9
BS13seqs = BS_info$BS_seq[match(TF13,BS_info$TF)]
print(BS13seqs)
print(min(nchar(BS13seqs)))

# 4p8 sequences in promoters at the mutaiton location
dpe = 'TCCAGCACCTGCCCCCG'
inr = 'GAGCCTGCCTTCCCGCCG'
caat = 'GCCGCGATTGGCACAGCC'
gc1 = 'CCGCGTTCATGGCCTTAAAGGGACAGCGCCCCGCGCCCGCCCCGCCCCGCCCCGGCCCGCCCC'
gc15 = 'CCGCGTTCATGGCCTTAAAGGGACAGCGCCCCGCGCCATCCTAGCCCTCTCCCGGTTCGATCC'
tata= 'TCACAGCCGTTTTATACCGGCCCG'
tatabreu='TCACAGCCGTTTTATAGGCGTCCCC'
del_jct = 'GCCCGCGCCGCCCGTGGCCATCGCA'
wt_jct = 'GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGG'

# 5p1 and 5p7 info files
BS_v7 = read.csv('/path_to_syn_enh_info_file/BS_info_v7_constructs_2024_05_16.csv',header=T)

BS_v7$TF = sapply(BS_v7$insert,function(x){strsplit(x,split="v7ins_")[[1]][2]})
BS_v7$seqj = sapply(BS_v7$seq,function(x){paste(strsplit(x,split="  ")[[1]],collapse='')})
BS_v7$seqj = sapply(BS_v7$seqj,function(x){paste(strsplit(x,split=" ")[[1]],collapse='')})
BS_v7$seqj = sapply(BS_v7$seqj,function(x){paste(strsplit(x,split=" ")[[1]],collapse='')})
BS_v7$BS_seq = sapply(BS_v7$seqj,function(x){strsplit(strsplit(x,split="ACTGGACTGC")[[1]][2],split='GCCTGGAG')[[1]][1]})
BS_v7$BS_len = sapply(BS_v7$BS_seq,nchar)
BS_v7$BS_seqrc = as.character(reverseComplement(DNAStringSet(BS_v7$BS_seq)))
print(head(BS_v7))

  No    TF
1  1 FOSL1
2  2 MEF2B
3  3  NFIA
4  4 NFKB1
5  5 MEF2A
6  6 RUNX1
                                                                                          seq
1     CACAATGTGGGGGGTGGGGGAGAGAGAAAAATCAAA GGATGAGTCACC AATGTTCAGAAAAAAGAGGTTTTCAAAAAGGAAGTGG
2   CACAATGTGGGGGGTGGGGGAGAGAGAAAAATCAAA GTTGCTATTTTTGG AATGTTCAGAAAAAAGAGGTTTTCAAAAAGGAAGTGG
3  CACAATGTGGGGGGTGGGGGAGAGAGAAAAATCAAA TTGGCACGGTGCCAA AATGTTCAGAAAAAAGAGGTTTTCAAAAAGGAAGTGG
4      CACAATGTGGGGGGTGGGGGAGAGAGAAAAATCAAA GGGAAATTCCC AATGTTCAGAAAAAAGAGGTTTTCAAAAAGGAAGTGG
5 CACAATGTGGGGGGTGGGGGAGAGAGAAAAATCAAA ACAGCTAAAAATAGAA AATGTTCAGAAAAAAGAGGTTTTCAAAAAGGAAGTGG
6      CACAATGTGGGGGGTGGGGGAGAGAGAAAAATCAAA GAAACCACAGA AATGTTCAGAAAAAAGAGGTTTTCAAAAAGGAAGTGG
            BS_seq
1     GGATGAGTCACC
2   GTTGCTATTTTTGG
3  TTGGCACGGTGCCAA
4      GGGAAATTCCC
5 ACAGCTAAAAATAGAA
6      GAAACCACAGA
[1] "ACATCTAAAATCAAAATAAAC" "ACATCTAAAAATAAAC"      "GTTGCTATTTTTGG"       
[4] "AGAGGAAGTGA"           "AGTGAAAGTGAAAGT"       "GGG

### Lib 3.8

In [ ]:
%%R
# match directly to the actual BS seqs

u38f = lapply(m38f$TF_sequence,function(x){
    s=strsplit(x,split="-")[[1]]
    sm = s[which.max(nchar(s))]
    if (nchar(sm)>=11){v=as.character(Views(sm,1:(nchar(sm)-10),11:nchar(sm)))
        u=TF8[unique(unlist(lapply(v,function(y){grep(y,BS8seqs)})))]}
    if (nchar(sm)<11){u=TF8[unique(grep(sm,BS8seqs))]}
    return(u)})

print(head(u38f))



In [156]:
%%R
l38tf = sapply(u38f,length)
te=rep('no',length(l38tf))
te[l38tf==1]=unlist(u38f[l38tf==1])
print(table(te))
m38f$matchedTF = te # does the TF determined by alignment match the actual TFBS sequence
print(head(m38f))
write.csv(m38f,file='/outpath/lib3p8_BC_TF_mapping_table_filtered_by_14bpBCseq_and_adding_TFmatchinfo_kmer11_2024_05_31.csv',row.names=F)


te
   ALT   CTCF  FOSL1   IRF2  MEF2B  NFKB1    PU1    REF     no 
 93510  32322  78162  73807  85286  89101  98404  93708 109298 
   Reference                            Read_name    BC_sequence
2        REF e366d944-cbcb-4f86-b5fb-4f8482af514b GAACCCATCTCGTG
6        REF 5d87c0f9-1c3d-4a90-90b4-9a92feb9fa8c TATGTTTTTCTAAC
8        REF 360c11cf-d9b9-48e0-bb2f-cb275b3d6e81 TAATAAGGGCCTTG
10       REF 286a6c48-9e5a-4e40-9b6a-8c1d1dc38b04 GCGAAGGAGGTGGA
12       REF 4532c036-cf50-40e4-9835-f9c273d5de59 ACTCAATGCAGCTT
13       REF 1cbdb4cb-4135-48f4-a774-25e46e7ecb99 GTCACGTCTGTGGA
              TF_sequence matchedTF
2  ACATCTAAAATCAAAATAAACA       REF
6  ACA--TAAAATCAAAATAAACA       REF
8  ACATCTAGAATCAAAATAAACA       REF
10 ACATATAAAATCAAAATAAACA       REF
12 ACATCTAAAATCAAAATAAACA       REF
13 ACATCTAAAATCAAAATAAACA       REF


### summarize table to retain unique BC-seq maps and remove BCs mapping to multiple seqs

In [ ]:
m38fk = m38f[m38f$Reference == m38f$matchedTF,]
print(dim(m38fk))
print(table(m38fk$Reference))

## count BC TF associations

BC_BSmap =tapply(m38fk$matchedTF,m38fk$BC_sequence,function(x){x})
print(head(BC_BSmap[[2]]))
uni = sapply(BC_BSmap,function(x){length(unique(x))})
print(table(uni))

BC_BSmap_single = do.call(rbind,lapply(BC_BSmap[uni==1],function(x){data.frame(table(x))}))
print(head(BC_BSmap_single))

Xok = lapply(BC_BSmap[uni>1],function(x){
                                 t=table(x)
                                to = t[order(-t)]
                                to[1]>=3*to[2]})

print(table(m38f$matchedTF, m38f$Reference))


BC_BSmap_X = do.call(rbind,lapply(BC_BSmap[uni>1][which(unlist(Xok))],function(x){
    t=table(x)
    d=data.frame(x=names(t)[which.max(t)],Freq=t[[which.max(t)]])
     return(d)
}))
                            
print(head(BC_BSmap_X))                                  
BC_BSmapf = rbind(BC_BSmap_X,BC_BSmap_single)
BC_BSmapf$BC = rownames(BC_BSmapf)

print(head(BC_BSmapf))
print(dim(BC_BSmapf))

write.csv(BC_BSmapf,'/outpath/lib3p8_BC_TF_nanopore_count_table_unique_BC_seq_and_3xover_secondarymatch_filtered_by_14bpBCseq_and_aligendReference_TFmatching_2024_06_07.csv',row.names=F)
  

## lib 3p9

make sure the 2 PU1 sites are getting corretly matched. They are identical for he first X bases

In [77]:
%%R
print(BS13seqs)

BS13_views = lapply(BS13seqs,function(x){
    n=nchar(x)
    if (n>11){s=as.character(Views(x,1:(nchar(x)-10),11:nchar(x)))}
    if (n<=11){s=x}
    return(s)})

BS13_views[[4]]

 [1] "TTGGCACGGTGCCAA"          "ACAGCTAAAAATAGAA"        
 [3] "GAAACCACAGA"              "AGAGGAAGTGAATAGAGGAAGTGA"
 [5] "AGAGGAAGTGAATTCACTTCCTCT" "GGGGGGCGGGGCC"           
 [7] "AAAAGAGGAAGTGAAAA"        "GAGGGCGTGGGCGGGGG"       
 [9] "GCGGGAAGCG"               "ACTTTCACTTTCACT"         
[11] "GAGTCATAATGAAACTG"        "GTTTATTTTTAGATGT"        
[13] "GAACCCGGAAGTGG"          
 [1] "AGAGGAAGTGA" "GAGGAAGTGAA" "AGGAAGTGAAT" "GGAAGTGAATA" "GAAGTGAATAG"
 [6] "AAGTGAATAGA" "AGTGAATAGAG" "GTGAATAGAGG" "TGAATAGAGGA" "GAATAGAGGAA"
[11] "AATAGAGGAAG" "ATAGAGGAAGT" "TAGAGGAAGTG" "AGAGGAAGTGA"


In [78]:
%%R 
# avoid PU1 2ff and 2fr double-matching
map39tf = lapply(m39f$TF_sequence,function(x){
    s=strsplit(x,split="-")[[1]]
    st=s[sapply(s,nchar)>4]
    sv=TF13[sapply(BS13_views,function(y){
       sum(sapply(y,function(i){length(grep(i,st))}))})>0]
    return(sv)})



[1] "NFIA"


In [85]:
%%R
m39f$TFmatch = sapply(1:length(m39f$Reference),function(x){m39f$Reference[x] %in% map39tf[[x]]})
print(head(m39f))
print(table(m39f$TFmatch))
# write the mapping with info on wether the aligned TF matches the actual TFBS seq
write.csv(m39f,file='/outpath/lib3p9_BC_TF_mapping_table_filtered_by_14bpBCseq_and_adding_TFmatchinfo_kmer10_2024_06_10.csv',row.names=F)


  Reference                            Read_name    BC_sequence
2      NFIA af238980-268e-46e1-bcf7-45e5b07c111a GAAAAAGGCGAAAA
3      NFIA a8d62edf-385b-4d9a-8686-ec2c71c558cb GTTTTTTGGCAAGA
4      NFIA 6b2319c0-9409-4590-a87f-74e209e73000 GGATATTCTGGCAA
5      NFIA 2a99fee5-8e8a-4b5e-8349-5f6c36989fa9 TGGCCGTGTGTGGG
6      NFIA 06693176-e58d-4d2d-b9b2-f566987704ae GGATGTTCTAGGGC
8      NFIA c50c0085-15af-465e-a425-5c7b06dc397a TCGGTTAGTAATCA
         TF_sequence TFmatch
2   -CGCCTAGGTGCCAAA   FALSE
3   TTGGCACGGTGCCAAA    TRUE
4   TTGGCACGGTGCCAAA    TRUE
5   TTGGCACGGTGCCAAA    TRUE
6 ATCGCCTA-GGTGCCAAA   FALSE
8   TTGGCACGGTGCCAAA    TRUE

 FALSE   TRUE 
 43824 186847 


In [86]:
%%R
## summarize count frquencies and remove non-unique maps
m39fk = m39f[m39f$TFmatch==TRUE,]
print(dim(m39fk))
print(table(m39fk$Reference))

## count BC TF associations

BC_BSmap =tapply(m39fk$Reference,m39fk$BC_sequence,function(x){x})
print(head(BC_BSmap[[2]]))
uni = sapply(BC_BSmap,function(x){length(unique(x))})
print(table(uni))

BC_BSmap_single = do.call(rbind,lapply(BC_BSmap[uni==1],function(x){data.frame(table(x))}))
print(head(BC_BSmap_single))

## BCs with higher frequenices that sometimes map to two seqs with 3x abundance filter
Xok = lapply(BC_BSmap[uni>1],function(x){
                                 t=table(x)
                                to = t[order(-t)]
                                to[1]>=3*to[2]})
print(head(Xok))


print(length(Xok))
print(length(unlist(Xok)))
print(table(unlist(Xok)))
BC_BSmap_X = do.call(rbind,lapply(BC_BSmap[uni>1][which(unlist(Xok))],function(x){
    t=table(x)
    d=data.frame(x=names(t)[which.max(t)],Freq=t[[which.max(t)]])
     return(d)
}))
                            
print(head(BC_BSmap_X))                                  
BC_BSmapf = rbind(BC_BSmap_X,BC_BSmap_single)
BC_BSmapf$BC = rownames(BC_BSmapf)

print(head(BC_BSmapf))
print(dim(BC_BSmapf))
print(table(BC_BSmapf$x))

# save all
write.csv(BC_BSmapf,'/outpath/lib3p9_BC_TF_nanopore_count_table_unique_BC_seq_and_3xover_secondarymatch_filtered_by_14bpBCseq_and_aligendReference_TFmatching_2024_06_10.csv',row.names=F)


NULL


## same for 4p8 lib
here we need to verify both the TF seq as well as the promoter mutations

In [17]:
%%R
print(head(m48f))
TF4 = TF8[c(1,2,4,6)]
BS4seqs = BS8seqs[c(1,2,4,6)]
print(dim(m48f))
print(TF4)
print(BS4seqs)

   Reference                            Read_name    BC_sequence
1 DPEmut_REF ecb3dfc3-a3e9-44a6-a16e-ccd4dfba370a CCGATTAGAGTGTA
2 DPEmut_REF a13e179d-fef6-43d7-bd70-59b00d77113b AAGCGGAATGAAGG
3 DPEmut_REF 230af6be-4dbf-4fd7-ba03-25c80ea91b7a TTAAGTTGACAATT
4 DPEmut_REF f3b04d64-8ab7-4986-ae51-9d5df0f92e8d GTTACTTGAGTATA
5 DPEmut_REF 48e73e0a-4d36-495a-b2d4-aba46b7e065e TCATGCTGCGTTCA
6 DPEmut_REF d11c8067-de38-42c9-a00d-2f94e9647b1a GCGGTTTGGTTTAT
              TF_sequence       ibtw_sequence matchedTF aligned_TF ibtw_match
1  ACATCTAAAATCAAAATAAACA  TCCAGCACCT-CCCCCGG       REF        REF         no
2  ACATCTAAAATCAAAATAAACA  TCCAGCACCTGCCCCCGG       REF        REF     DPEmut
3  ACATCTAAAATCAAAATAAACA  TCCAGCACCTGCCCCCGG       REF        REF     DPEmut
4  ACATCTAAAATCAAAATAAACA TGCCAGCACCTGCCCCC-G       REF        REF     DPEmut
5 ACATGTAAAATCAAAATGAAGCA  TCCAGCACCTGCCCCCGG       REF        REF     DPEmut
6  ACATCTAAAATCAAAATAAACA TCCAGCACTTGCCCCGAGG       REF        REF         no

In [18]:
%%R
# get unique matches
u48f = lapply(m48f$TF_sequence,function(x){
    s=strsplit(x,split="-")[[1]]
    sm = s[which.max(nchar(s))]
    if (nchar(sm)>=11){v=as.character(Views(sm,1:(nchar(sm)-10),11:nchar(sm)))
        u=TF4[unique(unlist(lapply(v,function(y){grep(y,BS4seqs)})))]}
    if (nchar(sm)<11){u=TF4[unique(grep(sm,BS4seqs))]}
    return(u)})

print(head(u48f))


[[1]]
[1] "REF"

[[2]]
[1] "REF"

[[3]]
[1] "REF"

[[4]]
[1] "REF"

[[5]]
[1] "REF"

[[6]]
[1] "REF"



In [22]:
%%R
l48tf = sapply(u48f,length)
te=rep('no',length(l48tf))
te[l48tf==1]=unlist(u48f[l48tf==1])
print(table(te))
m48f$matchedTF = te
m48f$aligned_TF = sapply(m48f$Reference,function(x){
    s=strsplit(x,split="_")[[1]]
    s[length(s)]})
print(head(m48f))

print(table(m48f$matchedTF ,m48f$aligned_TF))
print(head(m48f[grep('CGisl',m48f$Reference),]))

te
  ALT NFKB1   PU1   REF    no 
35029 39622 44654 35241 22472 
   Reference                            Read_name    BC_sequence
1 DPEmut_REF ecb3dfc3-a3e9-44a6-a16e-ccd4dfba370a CCGATTAGAGTGTA
2 DPEmut_REF a13e179d-fef6-43d7-bd70-59b00d77113b AAGCGGAATGAAGG
3 DPEmut_REF 230af6be-4dbf-4fd7-ba03-25c80ea91b7a TTAAGTTGACAATT
4 DPEmut_REF f3b04d64-8ab7-4986-ae51-9d5df0f92e8d GTTACTTGAGTATA
5 DPEmut_REF 48e73e0a-4d36-495a-b2d4-aba46b7e065e TCATGCTGCGTTCA
6 DPEmut_REF d11c8067-de38-42c9-a00d-2f94e9647b1a GCGGTTTGGTTTAT
              TF_sequence       ibtw_sequence matchedTF aligned_TF ibtw_match
1  ACATCTAAAATCAAAATAAACA  TCCAGCACCT-CCCCCGG       REF        REF         no
2  ACATCTAAAATCAAAATAAACA  TCCAGCACCTGCCCCCGG       REF        REF     DPEmut
3  ACATCTAAAATCAAAATAAACA  TCCAGCACCTGCCCCCGG       REF        REF     DPEmut
4  ACATCTAAAATCAAAATAAACA TGCCAGCACCTGCCCCC-G       REF        REF     DPEmut
5 ACATGTAAAATCAAAATGAAGCA  TCCAGCACCTGCCCCCGG       REF        REF     DPEmut
6  ACATCTAAA

### determine unique kmers that are specific to distinct promoter variations

In [18]:
%%R
# 4p8 sequences
dpe = 'ACCTGC'
inr = 'TGCCTT'
caat = 'GATTGGC'
gc1wt = 'CCGCCCCG' 
gc1 = 'TTCATG'
gc15 = 'CATCCTA'
tata= 'TTATACC'
tatabreu='TTATAGG'
del_jct = 'GCCCGTGGC'
wt_jct = 'GCCCGCCTT'

prom_seqs = c(dpe,inr,caat,gc1,gc1wt,gc15,tata,tatabreu,del_jct)
prom_names = c('DPEmut','INRmut','CAATflip','GCbox1mut','GCbox2wt','GCbox2mut','TATAins','TATABREUins','CpGislandDEL')
print(prom_seqs)
print(prom_names)

[1] "ACCTGC"    "TGCCTT"    "GATTGGC"   "TTCATG"    "CCGCCCCG"  "CATCCTA"  
[7] "TTATACC"   "TTATAGG"   "GCCCGTGGC"
[1] "DPEmut"       "INRmut"       "CAATflip"     "GCbox1mut"    "GCbox2wt"    
[6] "GCbox2mut"    "TATAins"      "TATABREUins"  "CpGislandDEL"


In [24]:
%%R
# verification
u48pf = lapply(m48f$ibtw_sequence,function(x){
    s=strsplit(x,split="-")[[1]]
    st=s[sapply(s,nchar)>5]
    p=prom_names[sapply(prom_seqs,function(x){length(grep(x,st))})>0]
    return(p)})

print(tail(u48pf))


[[1]]
[1] "CpGislandDEL"

[[2]]
character(0)

[[3]]
[1] "CpGislandDEL"

[[4]]
character(0)

[[5]]
[1] "CpGislandDEL"

[[6]]
[1] "CpGislandDEL"



In [27]:
%%R
# for different promoter variations we have to assign different matches
print(length(u48pf))
l48pf = sapply(u48pf,length)
te=rep('no',length(l48pf))
te[l48pf==1]=unlist(u48pf[l48pf==1])
te[l48pf>1]='ambiguous'
print(length(te))

gc = which(sapply(u48pf,function(x){length(grep('GCbox',x))})>=1)
print(head(gc[1:2]))
print(length(gc))
gcu48pf = u48pf[gc]
print(gcu48pf[100:104])
gclab = lapply(gcu48pf,function(x){
    if ('GCbox1mut' %in% x & 'GCbox2wt' %in% x){
       label = 'GCbox1mut'}
    if ('GCbox1mut' %in% x & 'GCbox2mut' %in% x ){
           label = 'GCbox12346mut'
       }
    if (('GCbox1mut' %in% x & !('GCbox2wt' %in% x) & !('GCbox2mut' %in% x)) | ('GCbox2wt' %in% x & !('GCbox1mut' %in% x)) | (('GCbox2mut' %in% x) & !('GCbox1mut' %in% x) ) ){
        label='no'}          
    return(label)})

print(gclab[100:102])
print(table(unlist(gclab)))

print(length(unlist(gclab)))

te[gc] = unlist(gclab)
print(table(te))


[1] 177018
[1] 177018
[1] 2599 2790
[1] 19173
[[1]]
[1] "GCbox1mut" "GCbox2wt" 

[[2]]
[1] "GCbox1mut" "GCbox2wt" 

[[3]]
[1] "GCbox1mut" "GCbox2wt" 

[[4]]
[1] "GCbox1mut" "GCbox2wt" 

[[5]]
[1] "GCbox1mut" "GCbox2wt" 

[[1]]
[1] "GCbox1mut"

[[2]]
[1] "GCbox1mut"

[[3]]
[1] "GCbox1mut"


GCbox12346mut     GCbox1mut            no 
         4007          9740          5426 
[1] 19173
te
     CAATflip  CpGislandDEL        DPEmut GCbox12346mut     GCbox1mut 
         5754         68695          9090          4007          9740 
       INRmut   TATABREUins       TATAins     ambiguous            no 
        10519          7945         18963           184         42121 


In [28]:
%%R
# create final alignment map and actual seq match table
m48f$ibtw_match = te
m48f$ibtw_aligned = sapply(m48f$Reference,function(x){strsplit(x,split="_")[[1]][1]})
print(head(m48f[m48f$ibtw_match=='ambiguous',]))
u48pf[[which(m48f$ibtw_match=='ambiguous')[100]]]
print(head(m48f))
print(table(m48f$ibtw_match,m48f$ibtw_aligned)) # look at how often they correspond

       Reference                            Read_name    BC_sequence
15580 INRmut_REF 8fb1e518-da5c-4ad4-a916-3542e8cc949c CTGCAGATTGTTTG
15658 INRmut_REF f5faaf94-c1e3-437d-8bee-287421dc9d42 ATTTCATCATCTGC
16333 INRmut_REF 15fbf5f7-8b08-4152-be39-cfb762ceaeb4 GTTGTATACATAGG
20384 INRmut_ALT e98461a1-0451-49e0-86f4-9057ed2f0346 GAAGTCATCCTGGA
21802 INRmut_ALT ebbb7345-30b8-4087-bcde-b18583774f58 CATACTGTTTCGTA
22112 INRmut_ALT 0c2dd704-aa0a-4ec3-8600-11bae773cc46 TACTAGGAGCGCCT
                 TF_sequence           ibtw_sequence matchedTF aligned_TF
15580 ACATCTAAAATCAAAATAAACA GGAAAGACCTGCCTTCCCGCCGC       REF        REF
15658 ACGT-TAAAATCAAAATAAACA   GAGACCTGCCTTCCCCGCCGC       REF        REF
16333 ACATCTAAAATCAAAATAAACA    -AACCTGCCTTTTCCGCCGC       REF        REF
20384    ACATAATCAAAATAAACCA    GAACCTGCCTTGCCGGCCGC       REF        ALT
21802      ACATCTAAAAATAAACA     GAACCTGCCTTCCCGCCGC       ALT        ALT
22112      ACATCTAAAAA-ATACA     GAACCTGCCTTCCCGCCGC       ALT        ALT

In [30]:
%%R

# write information to file 

write.csv(m48f,file='/outpath/lib4p8_BC_TF_mapping_table_filtered_by_14bpBCseq_and_adding_TFmatch_amd_promModi_info_kmer11_2024_06_07.csv',row.names=F)
print(head(m48f))
print(table(m48f$ibtw_match,m48f$ibtw_aligned))

   Reference                            Read_name    BC_sequence
1 DPEmut_REF ecb3dfc3-a3e9-44a6-a16e-ccd4dfba370a CCGATTAGAGTGTA
2 DPEmut_REF a13e179d-fef6-43d7-bd70-59b00d77113b AAGCGGAATGAAGG
3 DPEmut_REF 230af6be-4dbf-4fd7-ba03-25c80ea91b7a TTAAGTTGACAATT
4 DPEmut_REF f3b04d64-8ab7-4986-ae51-9d5df0f92e8d GTTACTTGAGTATA
5 DPEmut_REF 48e73e0a-4d36-495a-b2d4-aba46b7e065e TCATGCTGCGTTCA
6 DPEmut_REF d11c8067-de38-42c9-a00d-2f94e9647b1a GCGGTTTGGTTTAT
              TF_sequence       ibtw_sequence matchedTF aligned_TF ibtw_match
1  ACATCTAAAATCAAAATAAACA  TCCAGCACCT-CCCCCGG       REF        REF         no
2  ACATCTAAAATCAAAATAAACA  TCCAGCACCTGCCCCCGG       REF        REF     DPEmut
3  ACATCTAAAATCAAAATAAACA  TCCAGCACCTGCCCCCGG       REF        REF     DPEmut
4  ACATCTAAAATCAAAATAAACA TGCCAGCACCTGCCCCC-G       REF        REF     DPEmut
5 ACATGTAAAATCAAAATGAAGCA  TCCAGCACCTGCCCCCGG       REF        REF     DPEmut
6  ACATCTAAAATCAAAATAAACA TCCAGCACTTGCCCCGAGG       REF        REF         no

### count up BC to seq matching

In [32]:
%%R
print(head(m48f))
m48f$ibtw_match[grep('CpGislandDEL',m48f$ibtw_match)]='CGislandDEL'
m48f$ibtw_match[grep('TATABREUins',m48f$ibtw_match)]='TATABREins'
print(table(m48f$matchedTF,m48f$aligned_TF))
print(table(m48f$ibtw_match,m48f$ibtw_aligned))
print(dim(m48f))

# filter for perfect
m48fk = m48f[m48f$matchedTF == m48f$aligned_TF & m48f$ibtw_match==m48f$ibtw_aligned,]
print(dim(m48fk))
print(table(m48fk$ibtw_match,m48fk$ibtw_aligned))

## count BC TF associations

BC_BSmap =tapply(m48fk$Reference,m48fk$BC_sequence,function(x){x})
print(head(BC_BSmap[[2]]))
uni = sapply(BC_BSmap,function(x){length(unique(x))})
print(table(uni))

BC_BSmap_single = do.call(rbind,lapply(BC_BSmap[uni==1],function(x){data.frame(table(x))}))
print(head(BC_BSmap_single))

## BCs with higher frequenices that sometimes map to two seqs with 3x abundance filter
Xok = lapply(BC_BSmap[uni>1],function(x){
                                 t=table(x)
                                to = t[order(-t)]
                                to[1]>=5*to[2]})
print(head(Xok))


print(length(Xok))
print(length(unlist(Xok)))
print(table(unlist(Xok)))
BC_BSmap_X = do.call(rbind,lapply(BC_BSmap[uni>1][which(unlist(Xok))],function(x){
    t=table(x)
    d=data.frame(x=names(t)[which.max(t)],Freq=t[[which.max(t)]])
     return(d)
}))
                            
print(head(BC_BSmap_X))                                  
BC_BSmapf = rbind(BC_BSmap_X,BC_BSmap_single)
BC_BSmapf$BC = rownames(BC_BSmapf)

print(head(BC_BSmapf))
print(dim(BC_BSmapf))
print(table(BC_BSmapf$x))

write.csv(BC_BSmapf,'/outpath/lib4p8_BC_TF_nanopore_count_table_unique_BC_seq_and_3xover_secondarymatch_filtered_by_14bpBCseq_and_aligendReference_TF_IBTWmatching_2024_06_11.csv',row.names=F)


   Reference                            Read_name    BC_sequence
1 DPEmut_REF ecb3dfc3-a3e9-44a6-a16e-ccd4dfba370a CCGATTAGAGTGTA
2 DPEmut_REF a13e179d-fef6-43d7-bd70-59b00d77113b AAGCGGAATGAAGG
3 DPEmut_REF 230af6be-4dbf-4fd7-ba03-25c80ea91b7a TTAAGTTGACAATT
4 DPEmut_REF f3b04d64-8ab7-4986-ae51-9d5df0f92e8d GTTACTTGAGTATA
5 DPEmut_REF 48e73e0a-4d36-495a-b2d4-aba46b7e065e TCATGCTGCGTTCA
6 DPEmut_REF d11c8067-de38-42c9-a00d-2f94e9647b1a GCGGTTTGGTTTAT
              TF_sequence       ibtw_sequence matchedTF aligned_TF ibtw_match
1  ACATCTAAAATCAAAATAAACA  TCCAGCACCT-CCCCCGG       REF        REF         no
2  ACATCTAAAATCAAAATAAACA  TCCAGCACCTGCCCCCGG       REF        REF     DPEmut
3  ACATCTAAAATCAAAATAAACA  TCCAGCACCTGCCCCCGG       REF        REF     DPEmut
4  ACATCTAAAATCAAAATAAACA TGCCAGCACCTGCCCCC-G       REF        REF     DPEmut
5 ACATGTAAAATCAAAATGAAGCA  TCCAGCACCTGCCCCCGG       REF        REF     DPEmut
6  ACATCTAAAATCAAAATAAACA TCCAGCACTTGCCCCGAGG       REF        REF         no

# Library 5p1 

In [14]:
%%R
m51f$Reference[m51f$Reference=='ref']='REF'
m51f$Reference[m51f$Reference=='alt']='ALT'
print(m51f[200000:200005,])
print(table(m51f$Reference))

       Reference                            Read_name  BC_sequence
354207     MEF2A 72cf9719-6ceb-4e4a-92fc-1dcf6b78b596 TATGTAGCATTT
354209     MEF2A 68c8aa8d-6a50-4314-92e9-b3f61a0171c4 TCTAGTCGGGAG
354210     MEF2A e32da65b-ecfa-4627-9992-4d13171c2153 CCCTCATAGGTT
354211     MEF2A fec235f6-ee0e-4b19-b551-386d92700743 TGGAAAATCGTG
354212     MEF2A 5fab707e-1856-40d8-9ba2-a6bfc3f3bd48 ATCGAATGCAAG
354214     MEF2A e62d7a0b-f954-42d2-b0c3-0a618be6dca1 CTCGGGTCGCTC
            TF_sequence               ibtw_sequence
354207   GCTAAAAATAGAAG CGCCGCCCGCCTTTTAAAGGGACAGCG
354209   GCTAAAAATAGAAG   CGCCGCCCGCCTTAAAGGGACAGCG
354210  GCTAAAATTGAG-AG   CGCCGCCCGCCTTAAAGGGACAGCG
354211   GCTAAAAATAGAAG   CGCCGCCCGCCTTAAAGGGACAGCG
354212 CCTAGAAAAATAGAAG   CGCCGCCCGCCTTAAAGGGACAGC-
354214   GCTAAAAATAGAAG   CGCCGCCCGCC-TAAAGGGACAGCG

        AHR_PU1_TF65                  ALT                BACH1 
                6173                 6491                 9882 
       BCH1_PU1_BCH1     CGisland_del_

In [52]:
%%R
gwt=grep(wt_jct ,m51f$ibtw_sequence)
gdel=grep(del_jct, m51f$ibtw_sequence)
print(length(intersect(gwt,gdel)))
gwtk = setdiff(gwt,gdel)
gdelk = setdiff(gdel,gwt)
print(length(intersect(gdelk,gwtk)))
m51f$ibassigned = NA
m51f$ibassigned[gwtk] = 'wt'
m51f$ibassigned[gdelk] = 'CGislandDEL'

print(head(m51f))


[1] 2
[1] 0
  Reference                            Read_name  BC_sequence
1       REF 9c8aec61-298d-47df-8ada-29012375d9fc AAACTTGTCCTT
2       REF d3441c67-c0a8-4b68-9a40-db70a0f88816 AGTGGCTACTGT
3       REF 81957966-7ce8-4c5f-bb27-11f69cfdd469 GCGTTGTTGGGG
5       REF c5f26f4b-2582-411a-85a6-fc0738b1ae62 AAATAGGTTGAG
7       REF db9c0968-2c33-4a99-9eaf-48c8248586b1 CTATGGCGATGG
9       REF ce1576bd-3d68-47a3-a7ae-6f4c49659747 GGTACGGCTCGA
             TF_sequence               ibtw_sequence ibassigned
1 ACATCTAAAATCAAAATAAACA CGCCCCTGCCACCTTAAAGGGACAGCG       <NA>
2 ACATCTAAAATCAAAATAAACA   CGCCGCCCGCCTTAAAGGGACAGCG         wt
3 ACATCTAAAATCAAAATAAACA   CGCCGCCCGCCTTAAAGGGACAGCG         wt
5 ACATCTAAAATCAAAATAAACA   CGCCGCCCGCCTTAAAGGGACAGCG         wt
7 ACATCTAAAATGAAAATAAACA   CGCCGCCCGCCTTAAAGGGACAGCG         wt
9 ACATCTAAAATCAAAATAAACA   CGCCGCCCGCCTTAAAGGGACAGCG         wt


In [84]:
%%R
# assign TF sequence

TF_map = data.frame(libname = unique(m51f$Reference),tfname=unique(m51f$Reference))
TF_map$tfname[grep('CGislan',TF_map$libname)]=sapply(TF_map$libname[grep('CGislan',TF_map$libname)],function(x){
    s=strsplit(x,split="_")[[1]]
    paste(s[3:length(s)],collapse="_")})
print(head(TF_map))
m51f$tfname = TF_map$tfname[match(m51f$Reference,TF_map$libname)]
print(unique(m51f$tfname))
print(head(m51f[m51f$tfname=='enh_del',]))

print(head(BS8seqs))
print(TF8)

# all BS seqs for 5.1
seqs51 = c(BS_v7$BS_seqrc[BS_v7$in.5p1=='y'][1:25],BS8seqs[c(1,2,4,6)],'GGCGCAG')
print(length(seqs51))
print(seqs51)

# and corresponding TF names
TF51 = c(BS_v7$TF[BS_v7$in.5p1=='y'][1:25],TF8[c(1,2,4,6)],'enh_del')
print(TF51)

             libname tfname
1                REF    REF
2                ALT    ALT
3   CGisland_del_REF    REF
4   CGisland_del_ALT    ALT
5   CGisland_del_PU1    PU1
6 CGisland_del_NFKB1  NFKB1
 [1] "REF"                "ALT"                "PU1"               
 [4] "NFKB1"              "enh_del"            "FX1_DBP_FX1"       
 [7] "CTCF_in"            "CTCF_out"           "FX1"               
[10] "DBP"                "BACH1"              "MEF2A"             
[13] "IRF2"               "SPI1"               "RUNX3"             
[16] "RELB"               "FX1_BCH1_IRF2"      "IRF_PU1_RELB"      
[19] "BCH1_PU1_BCH1"      "MF2A_RELB_MF2A"     "MF2A_PU1_MF2A"     
[22] "MF2A_SPI1_FX1_IRF2" "MF2A_SPI1_FX1_BH1"  "MF2A_PU1_CTCFout"  
[25] "MF2A_PU1_CTCFin"    "CTCFout_MF2A_PU1"   "AHR_PU1_TF65"      
[28] "MF2A_PU1_TF65"      "RUNX_PU1_RUNX"      "MF2A_MF2A_MF2A"    
                 Reference                            Read_name  BC_sequence
74012 CGisland_del_enh_del 8033ceb4-07ae-4a07-8

In [102]:
%%R
print(head(m51f[c(2,10000,30000,100000),]))

# perform manual matching
u51pf = lapply(m51f$TF_sequence,function(x){
    s=strsplit(x,split="-")[[1]]
    st=s[sapply(s,nchar)>6]
    ust=unlist(lapply(st,function(x){
           m=min(10,nchar(x))
           as.character(Views(x,1:(nchar(x)-(m-1)),m:nchar(x)))}))
    p=table(unlist(lapply(ust,function(x){grep(x,seqs51)})))
    n=TF51[as.numeric(names(p))]
    return(n)})

print(head(u51pf))

                  Reference                            Read_name  BC_sequence
2                       REF d3441c67-c0a8-4b68-9a40-db70a0f88816 AGTGGCTACTGT
21822                   ALT b59bdb54-c905-41a2-820b-efcb8ada2a2e GGAGCCAGGGTC
82155  CGisland_del_enh_del 66660caf-5fc7-481f-8544-227013e3f0c0 ATTTTAAATAAT
174050 CGisland_del_enh_del cc58d008-9ca5-4989-8133-d556887ac7f3 CGGTAGGGTGAG
                  TF_sequence              ibtw_sequence  ibassigned  tfname
2      ACATCTAAAATCAAAATAAACA  CGCCGCCCGCCTTAAAGGGACAGCG          wt     REF
21822  ACATCTAAAAATAAACAATGTT  CGCCGCCCGCC---AA-GGACAGCG        <NA>     ALT
82155                 GGCGCAG GCCCGCGCCGCCCGTGGCCATCGCAA CGislandDEL enh_del
174050                GGCGCAG GCCCGCGCCGCCCGTGGCCATCGCAA CGislandDEL enh_del
[[1]]
[1] "REF" "ALT"

[[2]]
[1] "REF" "ALT"

[[3]]
[1] "REF" "ALT"

[[4]]
[1] "REF" "ALT"

[[5]]
[1] "REF" "ALT"

[[6]]
[1] "REF" "ALT"



In [103]:
%%R
# check if the aligned tfname is in the extracted tf name
m51f$TFseq_in_tfname = sapply(1:length(m51f[,1]),function(x){m51f$tfname[x] %in% u51pf[[x]]})
print(head(m51f))
print(table(m51f$TFseq_in_tfname))

  Reference                            Read_name  BC_sequence
1       REF 9c8aec61-298d-47df-8ada-29012375d9fc AAACTTGTCCTT
2       REF d3441c67-c0a8-4b68-9a40-db70a0f88816 AGTGGCTACTGT
3       REF 81957966-7ce8-4c5f-bb27-11f69cfdd469 GCGTTGTTGGGG
5       REF c5f26f4b-2582-411a-85a6-fc0738b1ae62 AAATAGGTTGAG
7       REF db9c0968-2c33-4a99-9eaf-48c8248586b1 CTATGGCGATGG
9       REF ce1576bd-3d68-47a3-a7ae-6f4c49659747 GGTACGGCTCGA
             TF_sequence               ibtw_sequence ibassigned tfname
1 ACATCTAAAATCAAAATAAACA CGCCCCTGCCACCTTAAAGGGACAGCG       <NA>    REF
2 ACATCTAAAATCAAAATAAACA   CGCCGCCCGCCTTAAAGGGACAGCG         wt    REF
3 ACATCTAAAATCAAAATAAACA   CGCCGCCCGCCTTAAAGGGACAGCG         wt    REF
5 ACATCTAAAATCAAAATAAACA   CGCCGCCCGCCTTAAAGGGACAGCG         wt    REF
7 ACATCTAAAATGAAAATAAACA   CGCCGCCCGCCTTAAAGGGACAGCG         wt    REF
9 ACATCTAAAATCAAAATAAACA   CGCCGCCCGCCTTAAAGGGACAGCG         wt    REF
  TFseq_in_tfname
1            TRUE
2            TRUE
3            TR

In [108]:
%%R
m51ff=m51f[m51f$TFseq_in_tfname==T & !is.na(m51f$ibassigned),]
print(dim(m51ff))
print(table(m51ff$Reference))
print(table(m51f$Reference))

write.csv(m51ff,'/outpath/BC_map_VG_map_filtered_ibtw_and_TF_checked_by_JK_2024_07_30.csv')



[1] 315733      8

        AHR_PU1_TF65                  ALT                BACH1 
                4628                 4748                 5308 
       BCH1_PU1_BCH1     CGisland_del_ALT   CGisland_del_NFKB1 
                4075                 2322                 2325 
    CGisland_del_PU1     CGisland_del_REF CGisland_del_enh_del 
                2312                 1898                63543 
             CTCF_in             CTCF_out     CTCFout_MF2A_PU1 
                4448                 3960                 2848 
                 DBP                  FX1        FX1_BCH1_IRF2 
                8624                 9315                 4566 
         FX1_DBP_FX1                 IRF2         IRF_PU1_RELB 
                8395                 8298                20317 
               MEF2A       MF2A_MF2A_MF2A      MF2A_PU1_CTCFin 
               14520                 8329                 3228 
    MF2A_PU1_CTCFout        MF2A_PU1_MF2A        MF2A_PU1_TF65 
                3763 

In [110]:
%%R
## count BC TF associations

BC_BSmap =tapply(m51ff$Reference,m51ff$BC_sequence,function(x){x})

print(head(BC_BSmap[[2]]))
uni = sapply(BC_BSmap,function(x){length(unique(x))})
print(table(uni))

BC_BSmap_single = do.call(rbind,lapply(BC_BSmap[uni==1],function(x){data.frame(table(x))}))
print(head(BC_BSmap_single))

## BCs with higher frequenices that sometimes map to two seqs with 3x abundance filter
Xok = lapply(BC_BSmap[uni>1],function(x){
                                 t=table(x)
                                to = t[order(-t)]
                                to[1]>=3*to[2]})
print(head(Xok))


print(length(Xok))
print(length(unlist(Xok)))
print(table(unlist(Xok)))
BC_BSmap_X = do.call(rbind,lapply(BC_BSmap[uni>1][which(unlist(Xok))],function(x){
    t=table(x)
    d=data.frame(x=names(t)[which.max(t)],Freq=t[[which.max(t)]])
     return(d)
}))
                            
print(head(BC_BSmap_X))                                  
BC_BSmapf = rbind(BC_BSmap_X,BC_BSmap_single)
BC_BSmapf$BC = rownames(BC_BSmapf)

print(head(BC_BSmapf))
print(dim(BC_BSmapf))
print(table(BC_BSmapf$x))

write.csv(BC_BSmapf,'/outpath/lib5p1_BC_TF_nanopore_count_table_unique_BC_seq_and_3xover_secondarymatch_filtered_by_12bpBCseq_and_aligendReference_TF_IBTWmatching_2024_07_30.csv',row.names=F)


[1] "enh_del"
uni
    1     2     3     4     5 
73125  1195    74     9     2 
                                x Freq
AAAAAAAAGTTA CGisland_del_enh_del    1
AAAAAAACCGAC              enh_del    1
AAAAAAAGAAAC                  DBP   12
AAAAAAAGAAGT                 SPI1    8
AAAAAAATGTAC          FX1_DBP_FX1    1
AAAAAAGAAGAG    MF2A_SPI1_FX1_BH1   10
$AAACACGAGTTT
MF2A_PU1_CTCFin 
           TRUE 

$AAACATTGGGCA
CGisland_del_enh_del 
                TRUE 

$AAACGACACGTC
MF2A_SPI1_FX1_BH1 
             TRUE 

$AAAGAAACTGCG
MF2A_PU1_CTCFout 
            TRUE 

$AAAGACCTTGGA
IRF_PU1_RELB 
       FALSE 

$AAAGGAGGGTAG
MF2A_PU1_TF65 
         TRUE 

[1] 1280
[1] 1280

FALSE  TRUE 
  307   973 
                                x Freq
AAACACGAGTTT      MF2A_PU1_CTCFin   16
AAACATTGGGCA CGisland_del_enh_del   33
AAACGACACGTC    MF2A_SPI1_FX1_BH1    4
AAAGAAACTGCG     MF2A_PU1_CTCFout   11
AAAGGAGGGTAG        MF2A_PU1_TF65   42
AAAGGTCGAGAT        MF2A_PU1_MF2A   25
                             

## Lib 5p7

In [9]:
%%R


filter_table2= function(tab,BClen){
    fnbc = !(1:length(tab$BC_sequence) %in% grep('-',tab$BC_sequence)) & nchar(tab$BC_sequence)==BClen
    ftf = nchar(tab$TF_sequence)>6 |  tab$TF_sequence=='G' #enh_del should be 1bp and = G on
    tabk = tab[fnbc & ftf,]
    return(tabk)
}
print(head(m57))
m57f = filter_table2(m57,12)
print(head(m57f))
print(head(m57f[grep('IRF2_I',m57f$Reference),]))

  Reference                            Read_name    BC_sequence
1       ref 4e5917c7-a5fb-43d5-abee-d7dac5642cc8 CGTGTGGGAATGCA
2       ref 7d223233-422e-4c11-a782-7533fbece643   CAGGGTCCTGAG
3       ref fac8854d-6278-4600-8bc8-9f2ba7b88931   --CGTGGCGTAT
4       ref 6f788805-1600-4f83-8820-aa1218af6811   GTGTAGGAGGGG
5       ref b314856e-b870-46bb-b28c-ce7d529385a9   TTGTAGCAGGTG
6       ref fd19a940-2e0a-4cbd-8484-d0d2bd9edc91   GAGTCTGAGGCG
             TF_sequence                         ibtw_sequence
1 ACATTTAAAATCAAAATAAACA  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
2 ACATCTAAAATCAAAATAAACA  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
3 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCGCCGCCCGCCTTTAAAGGGA
4 ACATCTAAAATCAAAATAAACA  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
5 ACATCTAAAATCAAAATAAACA  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
6 ACATCTAAAATCAAAATAAACA  GCGCTCGGCCGGCCCGCG-CGCCCGCCTTAAAGGGA
  Reference                            Read_name  BC_sequence
2       ref 7d223233-422e-4c11-a782-7533fbece643 

In [172]:
%%R
m57f$Reference[m57f$Reference=='ref']='REF'
m57f$Reference[m57f$Reference=='alt']='ALT'
print(m57f[200000:200005,])
print(table(m57f$Reference))

        Reference                            Read_name  BC_sequence
284112 wt-CTCF_in 435ca956-eb66-4015-b08b-74f2e8ec165f GTTGGGATTATA
284113 wt-CTCF_in 18f0176a-46b8-4189-8ce3-dc33c3c861a8 GTCGCATAACAG
284115 wt-CTCF_in eec35d95-541e-4b4a-8dd2-1eb3558cc8e8 ATATAACTATCG
284116 wt-CTCF_in 057d71c8-60c3-4322-95ce-8b6642b45b9c GGGAGAAGGGTT
284117 wt-CTCF_in c3bf5882-d1cb-4fa8-b8aa-2b4c3145dc3f CGAGGCAGGACT
284118 wt-CTCF_in 5138b2b2-7612-4a71-85eb-0d6cd6adb3f0 AAGTCGGTCCTG
                 TF_sequence                        ibtw_sequence
284112  TGGCGCCCCCT-GTGGCCAG GCGCTCGGCCGGCCCGCGCCGCCCGCCTT-AAGGGA
284113 TGGCGCCCCC--GTGGCTGAG GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
284115 TGGCGCCCCCTTGGT-GCCAG GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
284116  TGGCGCCCCC-GCTGGCCAG GCGCTCGGCCGGCCCGCGCCGCCCGCCTT-AAGGGA
284117  TGG-GTCTCCT-GTGGCCAG GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
284118  TGGCGCCCCGTGGTGGCCAG GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA

                                 ALT              CGislandDel

In [173]:
%%R
tata= 'TTATACC'
del_jct = 'GCCCGTGGC'
wt_jct = 'GCCCGCCTT'

prom_seqs = c(tata,del_jct,wt_jct)
prom_names = c('TATAins','CpGislandDEL','wt')

gwt=grep(wt_jct ,m57f$ibtw_sequence)
gdel=grep(del_jct, m57f$ibtw_sequence)
gtata=grep(tata, m57f$ibtw_sequence)
print(length(intersect(gwt,gdel)))
print(length(intersect(gwt,gtata)))
print(length(intersect(gdel,gtata)))

gwtk = setdiff(setdiff(gwt,gdel),gtata)
gdelk = setdiff(setdiff(gdel,gwt),gtata)
gtatak=setdiff(setdiff(gtata,gwt),gdel)
print(length(intersect(gdelk,gwtk)))
m57f$ibassigned = NA
m57f$ibassigned[gwtk] = 'wt'
m57f$ibassigned[gdelk] = 'CGislandDEL'
m57f$ibassigned[gtatak] = 'TATAins'

print(head(m57f))
print(table(m57f$ibassigned))



[1] 53
[1] 95
[1] 16
[1] 0
  Reference                            Read_name  BC_sequence
2       REF 7d223233-422e-4c11-a782-7533fbece643 CAGGGTCCTGAG
4       REF 6f788805-1600-4f83-8820-aa1218af6811 GTGTAGGAGGGG
5       REF b314856e-b870-46bb-b28c-ce7d529385a9 TTGTAGCAGGTG
6       REF fd19a940-2e0a-4cbd-8484-d0d2bd9edc91 GAGTCTGAGGCG
7       REF 19b57c85-843e-44e5-9c09-39d094d3c8ef GACTATTCAACT
9       REF 2b06b442-e8f7-4d44-b64e-690e98bddd0e GTCTCGACCTAG
             TF_sequence                        ibtw_sequence ibassigned
2 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
4 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
5 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
6 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCG-CGCCCGCCTTAAAGGGA         wt
7 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCACAGCCCGCCT-----GGA       <NA>
9 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCGCCGCCCGCC-TAAAGGGA       <NA>

CGislandDEL     TATAins    

In [178]:
%%R
# assign TF sequence

m57f$tfname = sapply(m57f$Reference,function(x){
    s=strsplit(x,split="-")[[1]]
    if (length(s)==1){
        k=s}
    if (length(s)==2){
        k=s[2]}
    return(k)
})



 [1] "REF"                "ALT"                "PU1"               
 [4] "NFKB1"              "enh_del"            "FX1_DBP_FX1"       
 [7] "CTCF_in"            "CTCF_out"           "FX1"               
[10] "DBP"                "BACH1"              "MEF2A"             
[13] "IRF2"               "SPI1"               "RUNX3"             
[16] "RELB"               "FX1_BCH1_IRF2"      "IRF_PU1_RELB"      
[19] "BCH1_PU1_BCH1"      "MF2A_RELB_MF2A"     "MF2A_PU1_MF2A"     
[22] "MF2A_SPI1_FX1_IRF2" "MF2A_SPI1_FX1_BH1"  "MF2A_PU1_CTCFout"  
[25] "MF2A_PU1_CTCFin"    "CTCFout_MF2A_PU1"   "AHR_PU1_TF65"      
[28] "MF2A_PU1_TF65"      "RUNX_PU1_RUNX"      "MF2A_MF2A_MF2A"    


In [184]:
%%R


# all BS seqs for 57
seqs57 = c(BS_v7$BS_seqrc[c(1:29,31:47)],BS8seqs[c(1,2)],'G') # in the alignment it used on base for the enh deletion, which is a blank in the info file, so we add it here
print(length(seqs57))
print(seqs57)

# and corresponding TF names
TF57 = c(BS_v7$TF[c(1:29,31:47)],TF8[c(1,2)],'enh_del')
print(TF57)
print(min(sapply(seqs57[1:48],nchar)))

[1] 49
 [1] "TTCCTGTTTACTCATAGGTTATGTAACACATAGTTCCTGTTTACT"                              
 [2] "TGGCGCCCCCTGGTGGCCA"                                                        
 [3] "TGGCCACCAGGGGGCGCCA"                                                        
 [4] "TTCCTGTTTACTCATAGACTTTCGATATGACTCATCATAGTTCCTGTTTACT"                       
 [5] "ACTTTCGATATGACTCAT"                                                         
 [6] "ATGAGTCATATCGAAAGT"                                                         
 [7] "GTTATGTAACA"                                                                
 [8] "TGCTGAGTCATGC"                                                              
 [9] "GCTAAAAATAGAA"                                                              
[10] "GAAAGTGAAAGTGAAAGTAA"                                                       
[11] "TTTCACTTCCTCTTTTT"                                                          
[12] "CTATGAAACCACAAA"                                                          

In [190]:
%%R
print(head(m57f))
# check
u57pf = lapply(m57f$TF_sequence,function(x){
    s=strsplit(x,split="-")[[1]]
    st=s[sapply(s,nchar)>9]
    ust=unlist(lapply(st,function(x){
           m=10
           as.character(Views(x,1:(nchar(x)-(m-1)),m:nchar(x)))}))
    p=table(unlist(lapply(ust,function(x){grep(x,seqs57)})))
    n=TF57[as.numeric(names(p))]
    return(n)})

print(head(u57pf))

           Reference                            Read_name  BC_sequence
4                REF 6f788805-1600-4f83-8820-aa1218af6811 GTGTAGGAGGGG
12916            REF f8586d57-8201-4fc2-af36-6adc3fe2b336 ACTTTTAGAAGT
3863332 TATAins-IRF2 0f94f78b-3b73-45da-902e-fad31b782744 GGATCATCACTA
1282519     wt-RUNX3 74f53941-08e3-4a4c-9973-dd12f156d2bc GTACACAAATTT
                   TF_sequence                          ibtw_sequence
4       ACATCTAAAATCAAAATAAACA   GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA
12916   ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCCCGCGCCGCCCGCCTTAAAGGGA
3863332  GAAAGTGAAAGTGAAAGTAAG              TCACAGCCGTTTTATACCGGCCCGC
1282519       CTATGAAACCACAAAG  GCGCTCGGCCGGCCCGGCGCCGCCCGCCTTAAAGGGA
        ibassigned tfname
4               wt    REF
12916           wt    REF
3863332    TATAins   IRF2
1282519         wt  RUNX3
[[1]]
[1] "REF" "ALT"

[[2]]
[1] "REF" "ALT"

[[3]]
[1] "REF" "ALT"

[[4]]
[1] "REF" "ALT"

[[5]]
[1] "REF" "ALT"

[[6]]
[1] "REF" "ALT"



In [191]:
%%R
m57f$TFseq_in_tfname = sapply(1:length(m57f[,1]),function(x){m57f$tfname[x] %in% u57pf[[x]]})
# fix enh_del 
m57f$TFseq_in_tfname[grep('enh_del',m57f$tfname)] = m57f$TF_sequence[grep('enh_del',m57f$tfname)] == 'G'

print(head(m57f))
print(table(m57f$TFseq_in_tfname))




  Reference                            Read_name  BC_sequence
2       REF 7d223233-422e-4c11-a782-7533fbece643 CAGGGTCCTGAG
4       REF 6f788805-1600-4f83-8820-aa1218af6811 GTGTAGGAGGGG
5       REF b314856e-b870-46bb-b28c-ce7d529385a9 TTGTAGCAGGTG
6       REF fd19a940-2e0a-4cbd-8484-d0d2bd9edc91 GAGTCTGAGGCG
7       REF 19b57c85-843e-44e5-9c09-39d094d3c8ef GACTATTCAACT
9       REF 2b06b442-e8f7-4d44-b64e-690e98bddd0e GTCTCGACCTAG
             TF_sequence                        ibtw_sequence ibassigned tfname
2 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt    REF
4 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt    REF
5 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt    REF
6 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCG-CGCCCGCCTTAAAGGGA         wt    REF
7 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCACAGCCCGCCT-----GGA       <NA>    REF
9 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCGCCGCCCGCC-TAAAGGGA       <NA>    REF
  TFse

In [193]:
%%R
# simple mapping
m57ff=m57f[m57f$TFseq_in_tfname==T & !is.na(m57f$ibassigned),]
print(dim(m57ff))
print(table(m57ff$Reference))
print(table(m57f$Reference))
write.csv(m57ff,'/outpath/BC_map_5p7_VG_map_filtered_ibtw_and_TF_checked_by_JK_2024_07_30.csv')




[1] 5809876       8

                                 ALT              CGislandDel-AHR_AHR_AHR 
                               70856                                25467 
            CGislandDel-AHR_PU1_TF65             CGislandDel-AHR_TF65_AHR 
                               41641                                17957 
                   CGislandDel-BACH1                     CGislandDel-BATF 
                               28779                                29298 
           CGislandDel-BCH1_PU1_BCH1                  CGislandDel-CTCF_in 
                                9677                                 9417 
                CGislandDel-CTCF_out         CGislandDel-CTCFout_MF2A_PU1 
                               17750                                27240 
                     CGislandDel-DBP                      CGislandDel-FX1 
                               50484                                35217 
            CGislandDel-FX1_BATF_FX1            CGislandDel-FX1_BCH1_IRF2 
    

## Do one extra check to make sure all are mapped correctly
For 5p7 library


In [28]:
%%R
# reload
m57ff=read.csv('/outpath/BC_map_5p7_VG_map_filtered_ibtw_and_TF_checked_by_JK_2024_07_30.csv',header=T) # from earlier

print(dim(m57ff[m57ff$Reference=='wt-IRF2',])) # use example to look at table
print(head(m57ff))

[1] 65496     9
   X Reference                            Read_name  BC_sequence
1  2       REF 7d223233-422e-4c11-a782-7533fbece643 CAGGGTCCTGAG
2  4       REF 6f788805-1600-4f83-8820-aa1218af6811 GTGTAGGAGGGG
3  5       REF b314856e-b870-46bb-b28c-ce7d529385a9 TTGTAGCAGGTG
4  6       REF fd19a940-2e0a-4cbd-8484-d0d2bd9edc91 GAGTCTGAGGCG
5 10       REF a4b6622d-032d-48d0-89ad-03f4ee04e160 CGGACAAATGGG
6 12       REF 092b23b5-cbfd-4a1c-82f7-1bbd1ed1971d AGCCCAGCCACT
             TF_sequence                         ibtw_sequence ibassigned
1 ACATCTAAAATCAAAATAAACA  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
2 ACATCTAAAATCAAAATAAACA  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
3 ACATCTAAAATCAAAATAAACA  GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
4 ACATCTAAAATCAAAATAAACA  GCGCTCGGCCGGCCCGCG-CGCCCGCCTTAAAGGGA         wt
5 ACATCTAAAATCAAAATAAACA GCGCTCGGCCGGCCCGCGCCGCCCGCCTTTAAAGGGA         wt
6 ACATCTAAAATCAAAATAAACA  GCGCTCGGCCGGCCCTCGCCGCCCGCCTTAAAGGGA         wt
  tfname TF

In [71]:
%%R
# 5p1 and 5p7 lib info
BS_v7 = read.csv('/home/kribelba/updepla/users/kribelba/RMCE_results/RMCE_v7_5p1_artificial_enh_16to18cyc_NXT1009_2023_07_14/BS_info_v7_constructs_2024_05_16.csv',header=T)

BS_v7$TF = sapply(BS_v7$insert,function(x){strsplit(x,split="v7ins_")[[1]][2]})
BS_v7$seqj = sapply(BS_v7$seq,function(x){paste(strsplit(x,split="  ")[[1]],collapse='')})
BS_v7$seqj = sapply(BS_v7$seqj,function(x){paste(strsplit(x,split=" ")[[1]],collapse='')})
BS_v7$seqj = sapply(BS_v7$seqj,function(x){paste(strsplit(x,split=" ")[[1]],collapse='')})
BS_v7$BS_seq = sapply(BS_v7$seqj,function(x){strsplit(strsplit(x,split="ACTGGACTGC")[[1]][2],split='GCCTGGAG')[[1]][1]})
BS_v7$BS_seq_fl_l = sapply(1:length(BS_v7$seqj),function(x){as.character(reverseComplement(DNAStringSet(strsplit(BS_v7$seqj[x],split=BS_v7$BS_seq[x])[[1]][2])))})
BS_v7$BS_seq_fl_r = sapply(1:length(BS_v7$seqj),function(x){as.character(reverseComplement(DNAStringSet(strsplit(BS_v7$seqj[x],split=BS_v7$BS_seq[x])[[1]][1])))})
BS_v7$seqj_rc =  as.character(reverseComplement(DNAStringSet(BS_v7$seqj)))

# get the junctions between BS and surrounding seq
BS_v7$BS_len = sapply(BS_v7$BS_seq,nchar)
BS_v7$BS_seqrc = as.character(reverseComplement(DNAStringSet(BS_v7$BS_seq)))
BS_v7$BS_jct_l = as.character(subseq(DNAStringSet(BS_v7$seqj_rc),start=14,end = 27))
BS_v7$BS_jct_r = sapply(1:length(BS_v7$BS_len),function(x){as.character(subseq(DNAStringSet(BS_v7$seqj_rc[x]),start=20+BS_v7$BS_len[x]-6,end =20+BS_v7$BS_len[x]+7 ))})

print(length(unique(BS_v7$seqj_rc)))
print(length(unique(BS_v7$BS_jct_l)))
print(length(unique(BS_v7$BS_jct_r)))

#=
print(head(BS_v7))

[1] 47
[1] 20
[1] 22
              insert
1  v7ins_FX1_DBP_FX1
2      v7ins_CTCF_in
3     v7ins_CTCF_out
4 v7ins_FX1_BATF_FX1
5          v7ins_FX1
6         v7ins_BATF
                                                                                                  seq
1          CAACGTGGTACACTGGACTGC  AGTAAACAGGAACTATGTGTTACATAACCTATGAGTAAACAGGAA  GCCTGGAGCTGCAGACCATG
2                                    CAACGTGGTACACTGGACTGC  TGGCCACCAGGGGGCGCCA  GCCTGGAGCTGCAGACCATG
3                                   CAACGTGGTACACTGGACTGC  TGGCGCCCCCTGGTGGCCA   GCCTGGAGCTGCAGACCATG
4 CAACGTGGTACACTGGACTGC   AGTAAACAGGAACTATGATGAGTCATATCGAAAGTCTATGAGTAAACAGGAA   GCCTGGAGCTGCAGACCATG
5                                    CAACGTGGTACACTGGACTGC   ATGAGTCATATCGAAAGT  GCCTGGAGCTGCAGACCATG
6                                   CAACGTGGTACACTGGACTGC   ACTTTCGATATGACTCAT   GCCTGGAGCTGCAGACCATG
  in.5p1           TF
1      y  FX1_DBP_FX1
2      y      CTCF_in
3      y     CTCF_out
4      n FX1_BATF_FX1
5      y

In [73]:
%%R
# all BS seqs for 57
fl57_l = c(BS_v7$BS_jct_l[c(1:29,31:47)],'AATCAAAACATCTA','AATCAAAACATCTA','AGGCGCAG') # add ref, alt and enh-del manually
fl57_r = c(BS_v7$BS_jct_r[c(1:29,31:47)],'ATAAACAATGTTCA','CAATGTTCAGAAAA','AGGCGCAG')
print(length(fl57_r))
print(fl57_r)
# and corresponding TF names
TF57 = c(BS_v7$TF[c(1:29,31:47)],TF8[c(1,2)],'enh_del')
print(TF57)

[1] 49
 [1] "GTTTACTGCAGTCC" "GTGGCCAGCAGTCC" "GGCGCCAGCAGTCC" "GTTTACTGCAGTCC"
 [5] "GACTCATGCAGTCC" "CGAAAGTGCAGTCC" "TGTAACAGCAGTCC" "GTCATGCGCAGTCC"
 [9] "AATAGAAGCAGTCC" "AAAGTAAGCAGTCC" "TCTTTTTGCAGTCC" "CCACAAAGCAGTCC"
[13] "ATTTCCCGCAGTCC" "AAAGTAAGCAGTCC" "AAGCCCCGCAGTCC" "ATTTCCCGCAGTCC"
[17] "GTCATGCGCAGTCC" "AATAGAAGCAGTCC" "AATAGAAGCAGTCC" "GCGTGACGCAGTCC"
[21] "TGAAAGTGCAGTCC" "GAGTCATGCAGTCC" "GGCGCCAGCAGTCC" "GTGGCCAGCAGTCC"
[25] "AGTGAAAGCAGTCC" "TTTCCAGGCAGTCC" "TTTCCAGGCAGTCC" "GTGGTTTGCAGTCC"
[29] "AATAGAAGCAGTCC" "TTTCCAGGCAGTCC" "TTTCCAGGCAGTCC" "TTTCCAGGCAGTCC"
[33] "AGTGAAAGCAGTCC" "TTTCCAGGCAGTCC" "TTTCCAGGCAGTCC" "GTCATGCGCAGTCC"
[37] "TTTCCAGGCAGTCC" "TTTCCAGGCAGTCC" "AATAGAAGCAGTCC" "AGTGAAAGCAGTCC"
[41] "TGCGTGCGCAGTCC" "TGCGTGCGCAGTCC" "GAAAGTAGCAGTCC" "TTTCCAGGCAGTCC"
[45] "TTTCCAGGCAGTCC" "AATAGAAGCAGTCC" "ATAAACAATGTTCA" "CAATGTTCAGAAAA"
[49] "AGGCGCAG"      
 [1] "FX1_DBP_FX1"              "CTCF_in"                 
 [3] "CTCF_out"                 "FX1

In [116]:
%%R
# table with extra sequence context
em572 = read.table('/path_extra_alignment_seq_extraction/VG_20240910_5p7lib_extraseq_context/5p7_2.nomult.tsv',header=T,sep='\t')
print(head(em572))

m57ff$ext_TF_sequence=em572$TF_sequence[match(m57ff$Read_name,em572$Read_name)]

m57ff$TFid = sapply(m57ff$Reference,function(x){
    strsplit(x,split='-')[[1]][2]})

m57ff$TFid[m57ff$Reference=='REF'] = 'REF'
m57ff$TFid[m57ff$Reference=='ALT'] = 'ALT'


print(head(m57ff[m57ff$Reference=='REF',],n=8))




  Reference                            Read_name    BC_sequence
1       ref 4e5917c7-a5fb-43d5-abee-d7dac5642cc8 CGTGTGGGAATGCA
2       ref 7d223233-422e-4c11-a782-7533fbece643   CAGGGTCCTGAG
3       ref fac8854d-6278-4600-8bc8-9f2ba7b88931   --CGTGGCGTAT
4       ref 6f788805-1600-4f83-8820-aa1218af6811   GTGTAGGAGGGG
5       ref b314856e-b870-46bb-b28c-ce7d529385a9   TTGTAGCAGGTG
6       ref fd19a940-2e0a-4cbd-8484-d0d2bd9edc91   GAGTCTGAGGCG
                                 TF_sequence
1 AAAAAT-GAAACATTTAAAATCAAAATAAACAATGTTCAGAA
2 AAAAATCAAAACATCTAAAATCAAAATAAACAATGTTCAGAA
3 AAAAATCAAAACATCTAAAATCAAAATAAACAATGTTCAGAA
4 AAAAATCAAAACATCTAAAATCAAAATAAACAATGTTCAGAA
5 AAAAATCAAAACATCTAAAATCAAAATAAACAATGTTCAGAA
6 AAAAATCAAAACATCTAAAATCAAAATAAACAATGTTCAGAA
   X Reference                            Read_name  BC_sequence
1  2       REF 7d223233-422e-4c11-a782-7533fbece643 CAGGGTCCTGAG
2  4       REF 6f788805-1600-4f83-8820-aa1218af6811 GTGTAGGAGGGG
3  5       REF b314856e-b870-46bb-b28c-ce7

## matching flanks to extended TF seq

In [98]:
%%R
x=8
fl57_l[which(TF57==m57ff$Reference[x])]
m57ff$ext_TF_sequence[x]
length(grep(fl57_r[which(TF57==m57ff$Reference[x])],m57ff$ext_TF_sequence[x])==1

logical(0)


In [119]:
%%R
print(table(m57ff$TFid %in% TF57))
print(TF57)


   TRUE 
5809876 
 [1] "FX1_DBP_FX1"              "CTCF_in"                 
 [3] "CTCF_out"                 "FX1_BATF_FX1"            
 [5] "FX1"                      "BATF"                    
 [7] "DBP"                      "BACH1"                   
 [9] "MEF2A"                    "IRF2"                    
[11] "SPI1"                     "RUNX3"                   
[13] "RELB"                     "FX1_BCH1_IRF2"           
[15] "MF2c_IRF2_NFK2"           "IRF_PU1_RELB"            
[17] "BCH1_PU1_BCH1"            "MF2A_RELB_MF2A"          
[19] "MF2A_PU1_MF2A"            "PX5_IRF2_PX5"            
[21] "MF2A_SPI1_FX1_IRF2"       "MF2A_SPI1_FX1_BH1"       
[23] "MF2A_PU1_CTCFout"         "MF2A_PU1_CTCFin"         
[25] "CTCFout_MF2A_PU1"         "AHR_PU1_TF65"            
[27] "MF2A_PU1_TF65"            "RUNX_PU1_RUNX"           
[29] "MF2A_MF2A_MF2A"           "IRF_PU1_TF65"            
[31] "RUNX3_PU1_TF65"           "TF65"                    
[33] "PU1_PU1_PU1"              "MF2A

In [ ]:
%%R
fldf=do.call(rbind,lapply(1:length(m57ff[,1]),function(x){
    data.frame(lfl_match = length(grep(fl57_l[which(TF57==m57ff$TFid[x])],m57ff$ext_TF_sequence[x]))==1,
               rfl_match = length(grep(fl57_r[which(TF57==m57ff$TFid[x])],m57ff$ext_TF_sequence[x]))==1)}))
print(tail(fldf))
m57fff = cbind(m57ff,fldf)
print(head(m57fff[m57fff$Reference=='wt-enh_del']))

In [ ]:
%%R
write.csv(m57fff,'/outpath/BC_map_5p7_VG_map_filtered_ibtw_and_TF_and_flank_left_right_checked_by_JK_2024_09_11.csv',row.names=F)



In [195]:
%%R
## count BC TF associations

BC_BSmap =tapply(m57ff$Reference,m57ff$BC_sequence,function(x){x})

print(head(BC_BSmap[[2]]))
uni = sapply(BC_BSmap,function(x){length(unique(x))})
print(table(uni))

BC_BSmap_single = do.call(rbind,lapply(BC_BSmap[uni==1],function(x){data.frame(table(x))}))
print(head(BC_BSmap_single))

## BCs with higher frequenices that sometimes map to two seqs with 3x abundance filter
Xok = lapply(BC_BSmap[uni>1],function(x){
                                 t=table(x)
                                to = t[order(-t)]
                                to[1]>=3*to[2]})
print(head(Xok))


print(length(Xok))
print(length(unlist(Xok)))
print(table(unlist(Xok)))
BC_BSmap_X = do.call(rbind,lapply(BC_BSmap[uni>1][which(unlist(Xok))],function(x){
    t=table(x)
    d=data.frame(x=names(t)[which.max(t)],Freq=t[[which.max(t)]])
     return(d)
}))
                            
print(head(BC_BSmap_X))                                  
BC_BSmapf = rbind(BC_BSmap_X,BC_BSmap_single)
BC_BSmapf$BC = rownames(BC_BSmapf)

print(head(BC_BSmapf))
print(dim(BC_BSmapf))
print(table(BC_BSmapf$x))

write.csv(BC_BSmapf,'/outpath/lib5p7_BC_TF_nanopore_count_table_unique_BC_seq_and_3xover_secondarymatch_filtered_by_12bpBCseq_and_aligendReference_TF_IBTWmatching_2024_08_03.csv',row.names=F)


                                      x Freq
AAAAAAAAAGAT                 wt-enh_del   10
AAAAAAAACCTC                     wt-FX1    1
AAAAAAAACTAC CGislandDel-RUNX3_PU1_TF65    1
AAAAAAAAGAAT                 wt-enh_del    2
AAAAAAAATAAC       TATAins-FX1_BATF_FX1    1
AAAAAAAATCGA                 wt-CTCF_in    2
$AAAAAAAAGGCA
TATAins-FX1_BATF_FX1 
                TRUE 

$AAAAAAGAAAGA
CGislandDel-IRF_PU1_RELB 
                   FALSE 

$AAAAAATGGTGC
wt-IRF2_IRF2_IRF2 
             TRUE 

$AAAAACATATAA
CGislandDel-MF2A_PU1_TF65 
                     TRUE 

$AAAAACCGATCT
TATAins-AHR_PU1_TF65 
               FALSE 

$AAAAACCTGTAG
wt-RELB 
   TRUE 

[1] 58114
[1] 58114

FALSE  TRUE 
19630 38484 
                                      x Freq
AAAAAAAAGGCA       TATAins-FX1_BATF_FX1    6
AAAAAATGGTGC          wt-IRF2_IRF2_IRF2    4
AAAAACATATAA  CGislandDel-MF2A_PU1_TF65   46
AAAAACCTGTAG                    wt-RELB   24
AAAAACGGTGAG CGislandDel-RUNX3_PU1_TF65   20
AAAAACGTGTTG      wt-MF2A_SP

## lib 5p8

In [204]:
%%R
m58f$Reference[m58f$Reference=='ref']='REF'
m58f$Reference[m58f$Reference=='alt']='ALT'
print(m58f[200000:200005,])
print(table(m58f$Reference))

        Reference                            Read_name  BC_sequence
270357 wt-enh_del 48399d60-98ae-45d2-9ce6-6287c7b67308 TTGTTATGCGCG
270359 wt-enh_del bc84cf35-0218-4a45-9fbd-af494f88bd52 GACGCTTAACTG
270361 wt-enh_del 96fc8ddb-ba38-4a44-bde6-cffd402ba147 TTTTGTGGTCAG
270362 wt-enh_del 0486c11c-c515-4804-a775-d0b54501d76b TTTTGTGTTTCG
270364 wt-enh_del 633cca70-a170-411c-a95c-9d2221a8dd18 CGTCGTGGGGCC
270365 wt-enh_del c9884626-1e6e-46e2-88e1-940a13f14b85 TAACTGAAAACA
                                                              TF_sequence
270357                                                                  G
270359                                                   TCCTCACTGACAACTG
270361 CAATTAATTCATGTATCATTGCAGCATTTGATTTTACTTTCACTGCTTCAGTGACTGAACTCACTG
270362                                                                  G
270364                            CAACTGGTCATGTAGTAACAACTTCAAGCTGTTTCCAGG
270365                                                                  G
      

In [208]:
%%R
# confirm that ibtw is wt
wt_jct = 'GCCCGCCTT'

gwt=grep(wt_jct ,m58f$ibtw_sequence)
print(length(m58f$ibtw_sequence))
print(length(gwt))

m58f$ibassigned = NA
m58f$ibassigned[gwt] = 'wt'

print(head(m58f))
print(table(m58f$ibassigned))


[1] 2977899
[1] 2656908
  Reference                            Read_name  BC_sequence
1       REF 91e3482a-c5b8-410a-9547-58b118de5e71 GCTACGCTCCTT
2       REF e8839302-c063-4545-9ef1-6a8556e1d04e GAACCGTACGGC
3       REF d6538daf-91b6-441a-99e7-43661c04fa53 GAAGATGGGCTA
4       REF 4884527c-9787-427f-9a7e-09dd47ed96fd TCCACCTCATCG
5       REF d601abea-9e73-4f8b-90d2-3f53f6d038a1 CGTGATATGGAG
6       REF b3c19ab5-3bb3-42b6-a562-533ff0e2c372 GAATGCGCGAGG
              TF_sequence                           ibtw_sequence ibassigned
1 ACATCTAAAATCAGAATAGAACA GCG-TCGGCCAAGCCACCGCGCCGCCCGCCTTAAAGGGA         wt
2  ACATCTAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
3  ACATCTAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
4  ACAT-TAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
5  ACATCTAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
6  ACATCTAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt

  

In [239]:
%%R
# check TF assignment
fasta_5p8 = readDNAStringSet('path/5p8_library_seqeunces_12bp_for_nanopore_alignment_lox_to_lox_2024_05_16.fasta')
print(fasta_5p8[1:10])
le_flank = 'TTCGGTGCCCATGGTCTGCAGCTCCAGGC'
ri_flank = 'GCAGTCCAGTGTACCACGTTG'

enh_seqs_5p8_v7 = unlist(lapply(as.character(fasta_5p8[c(3:5,7:length(fasta_5p8))]),function(s){
    strsplit(strsplit(s,split=le_flank)[[1]][2],split=ri_flank)[[1]][1]}))
print(length(enh_seqs_5p8_v7))
print(length(names(enh_seqs_5p8_v7)))

enh_seqs_5p8 = c(BS8seqs[1:2],enh_seqs_5p8_v7)
print(length(enh_seqs_5p8))

enh_names_5p8 = c('REF','ALT',names(enh_seqs_5p8_v7))
print(head(enh_names_5p8))
print(length(enh_names_5p8))               
print(BS8seqs[1:2])


DNAStringSet object of length 10:
     width seq                                              names               
 [1]  2931 ATAACTTCGTATAATGTATGCTA...AGGATACTTTATACGAAGTTAT ref
 [2]  2926 ATAACTTCGTATAATGTATGCTA...AGGATACTTTATACGAAGTTAT alt
 [3]  2343 ATAACTTCGTATAATGTATGCTA...AGGATACTTTATACGAAGTTAT wt-MF2A_PU1_MF2A
 [4]  2349 ATAACTTCGTATAATGTATGCTA...AGGATACTTTATACGAAGTTAT wt-MF2A_SPI1_FX1_...
 [5]  2348 ATAACTTCGTATAATGTATGCTA...AGGATACTTTATACGAAGTTAT wt-MF2A_SPI1_FX1_BH1
 [6]  2290 ATAACTTCGTATAATGTATGCTA...AGGATACTTTATACGAAGTTAT wt-enh_del
 [7]  2490 ATAACTTCGTATAATGTATGCTA...AGGATACTTTATACGAAGTTAT 82895_GENERATED_G...
 [8]  2490 ATAACTTCGTATAATGTATGCTA...AGGATACTTTATACGAAGTTAT 42828_GENERATED_G...
 [9]  2490 ATAACTTCGTATAATGTATGCTA...AGGATACTTTATACGAAGTTAT 39615_GENERATED_G...
[10]  2490 ATAACTTCGTATAATGTATGCTA...AGGATACTTTATACGAAGTTAT 33460_GENERATED_G...
[1] 103
[1] 103
[1] 105
[1] "REF"                                        
[2] "ALT"                                        

In [263]:
%%R
# match

u58pf_1 = lapply(m58f$TF_sequence,function(x){
    s=strsplit(x,split="-")[[1]]
    st=s[sapply(s,nchar)>12]
    ust=unlist(lapply(st,function(x){
           m=13
           as.character(Views(x,1:(nchar(x)-(m-1)),m:nchar(x)))}))
    p=table(unlist(lapply(ust,function(x){grep(x,enh_seqs_5p8)})))
    n=enh_names_5p8[as.numeric(names(p))]
    return(n)})

print(head(u58pf_1))


[[1]]
[1] "REF"

[[2]]
[1] "REF"

[[3]]
[1] "REF"

[[4]]
[1] "REF"

[[5]]
[1] "REF"

[[6]]
[1] "REF"



In [264]:
%%R
m58f$TFseq_in_tfname = sapply(1:length(m58f[,1]),function(x){m58f$Reference[x] %in% u58pf_1[[x]]})
# fix enh_del 
m58f$TFseq_in_tfname[grep('enh_del',m58f$Reference)] = m58f$TF_sequence[grep('enh_del',m58f$Reference)] == 'G'

print(head(m58f))
print(table(m58f$TFseq_in_tfname))



  Reference                            Read_name  BC_sequence
1       REF 91e3482a-c5b8-410a-9547-58b118de5e71 GCTACGCTCCTT
2       REF e8839302-c063-4545-9ef1-6a8556e1d04e GAACCGTACGGC
3       REF d6538daf-91b6-441a-99e7-43661c04fa53 GAAGATGGGCTA
4       REF 4884527c-9787-427f-9a7e-09dd47ed96fd TCCACCTCATCG
5       REF d601abea-9e73-4f8b-90d2-3f53f6d038a1 CGTGATATGGAG
6       REF b3c19ab5-3bb3-42b6-a562-533ff0e2c372 GAATGCGCGAGG
              TF_sequence                           ibtw_sequence ibassigned
1 ACATCTAAAATCAGAATAGAACA GCG-TCGGCCAAGCCACCGCGCCGCCCGCCTTAAAGGGA         wt
2  ACATCTAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
3  ACATCTAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
4  ACAT-TAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
5  ACATCTAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
6  ACATCTAAAATCAAAATAAACA    GCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGA         wt
  TFseq_in_tfname
1        

In [ ]:
%%R
m58ff=m58f[m58f$TFseq_in_tfname==T & !is.na(m58f$ibassigned),]
print(dim(m58ff))
print(table(m58ff$Reference))
print(table(m58f$Reference))
write.csv(m58ff,'/outpath/BC_map_5p8_VG_map_filtered_ibtw_and_TF_checked_by_JK_2024_07_30.csv')

